In [1]:
# Import packages

import plotly.graph_objs as go
import pandas_bokeh
import pandas as pd
import numpy as np
import plotly

In [2]:
# Read in CSV, using GBK encoding to handle Chinese characters

df = pd.read_csv("AQI.csv", encoding= 'GBK')

In [3]:
# inspect a few rows

df.head(3)

,City_CH,City_EN,FID,Field_1,GbCity,GbProv,Prov_CH,Prov_EN,AQI_Mean_0601,AQI_Mean_0602,...,AQI_Std_0704,AQI_Std_0705,AQI_Std_0706,AQI_Std_0707,AQI_Std_0708,AQI_Std_0709,AQI_Std_0710,AQI_Std_0711,AQI_Std_0712,AQI_Std_0713
0,北京市,Beijing,0,24.0,1100,11,北京市,Beijing,48.625000,55.739130,...,4.808665,11.772069,2.377882,13.445815,17.290250,7.683745,13.425722,20.026372,23.046714,20.721324
1,天津市,Tianjin,1,245.0,1200,12,天津市,Tianjin,61.375000,73.652174,...,19.072186,22.338152,24.366543,19.384944,8.298769,9.974185,11.513277,14.588273,13.103686,10.678948
2,石家庄市,Shijiazhuang,2,228.0,1301,13,河北省,Hebei,64.041667,65.695652,...,19.723611,19.813541,4.082483,13.513111,16.516933,10.434235,11.002525,5.334473,16.150120,6.971669


In [4]:
# Right now each row contains AQI info for many different dates. 
# We need to trim down the columns to Mean only and reshape the DataFrame.

df.columns

Index(['City_CH', 'City_EN', 'FID', 'Field_1', 'GbCity', 'GbProv', 'Prov_CH',
       'Prov_EN', 'AQI_Mean_0601', 'AQI_Mean_0602',
       ...
       'AQI_Std_0704', 'AQI_Std_0705', 'AQI_Std_0706', 'AQI_Std_0707',
       'AQI_Std_0708', 'AQI_Std_0709', 'AQI_Std_0710', 'AQI_Std_0711',
       'AQI_Std_0712', 'AQI_Std_0713'],
      dtype='object', length=180)

In [5]:
# Make a list of every relevent column
cols = [col for col in list(df.columns) if 'AQI_Mean' in col]

# Tack on city and province, which we'll also keep
cols.extend(('City_CH', 'Prov_CH'))

In [6]:
# Filter down the DataFrame and inspect a few rows

df = df[cols]
df.head(3)

,AQI_Mean_0601,AQI_Mean_0602,AQI_Mean_0603,AQI_Mean_0604,AQI_Mean_0605,AQI_Mean_0606,AQI_Mean_0607,AQI_Mean_0608,AQI_Mean_0609,AQI_Mean_0610,...,AQI_Mean_0706,AQI_Mean_0707,AQI_Mean_0708,AQI_Mean_0709,AQI_Mean_0710,AQI_Mean_0711,AQI_Mean_0712,AQI_Mean_0713,City_CH,Prov_CH
0,48.625000,55.739130,145.500000,42.583333,44.000000,62.166667,87.666667,118.111111,70.391304,86.875000,...,27.111111,32.958333,64.217391,45.041667,63.500000,110.166667,116.375000,71.958333,北京市,北京市
1,61.375000,73.652174,169.750000,91.083333,45.666667,47.666667,99.458333,84.111111,80.217391,96.500000,...,55.777778,70.125000,51.000000,41.375000,70.166667,100.375000,76.041667,64.708333,天津市,天津市
2,64.041667,65.695652,92.416667,88.166667,62.000000,67.500000,97.500000,87.666667,94.043478,103.458333,...,47.333333,72.250000,67.130435,53.041667,59.166667,83.291667,57.416667,54.250000,石家庄市,河北省


In [7]:
# Use melt() to reshape the DataFrame and inspect a few rows

aqi_mean_df = df.melt(id_vars = ['City_CH', 'Prov_CH'])
aqi_mean_df.head(3)

,City_CH,Prov_CH,variable,value
0,北京市,北京市,AQI_Mean_0601,48.625000
1,天津市,天津市,AQI_Mean_0601,61.375000
2,石家庄市,河北省,AQI_Mean_0601,64.041667


In [8]:
# Parse out the date value into a separate column

aqi_mean_df['date_str'] = aqi_mean_df['variable'].str[9:]+'20'

In [9]:
# Convert our new date field into an actual datetime object

aqi_mean_df['date'] = pd.to_datetime(aqi_mean_df['date_str'])

In [10]:
# Drop our now meaningless "variable" column

aqi_mean_df.drop(columns='variable', inplace=True)

In [11]:
# Clean up our column headers

aqi_mean_df.columns = ['City', 'Province', 'Mean AQI by Day', 'Raw Date', 'Parsed Date']

In [12]:
# Review our cleaned up DataFrame!

aqi_mean_df.head()

,City,Province,Mean AQI by Day,Raw Date,Parsed Date
0,北京市,北京市,48.625000,060120,2020-06-01
1,天津市,天津市,61.375000,060120,2020-06-01
2,石家庄市,河北省,64.041667,060120,2020-06-01
3,唐山市,河北省,61.083333,060120,2020-06-01
4,秦皇岛市,河北省,60.958333,060120,2020-06-01


In [13]:
# Sum up all mean AQI measurements by city and make a separate DataFrame
total_aqi = pd.DataFrame(aqi_mean_df.groupby(['City'])['Mean AQI by Day'].sum()).reset_index()

# Clean up the columns
total_aqi.columns = ['City', 'Sum of Daily Mean AQIs']

# Trim down to the top 20 totals only
top_20 = total_aqi.sort_values(by='Sum of Daily Mean AQIs', ascending = False).head(20)

In [14]:
# Inspect a few rows

top_20.head(5)

,City,Sum of Daily Mean AQIs
73,和田地区,4652.500296
81,喀什地区,4533.994785
328,酒泉地区,3655.946256
83,嘉峪关市,3542.072233
174,晋城市,3317.625198


In [15]:
# Time to gear up for plotting. Lets set plotly to offline mode.

plotly.offline.init_notebook_mode(connected=True)

In [16]:
# Structure our data for a bar chart

data = [go.Bar(x=top_20['City'], y=top_20['Sum of Daily Mean AQIs'])]

In [17]:
# Create the plot!

plotly.offline.iplot(data, filename='Worst AQI by City 6/1 --> 7/13')

In [18]:
# We can also use Bokeh to do the same thing. Setting the output to notebook:

pandas_bokeh.output_notebook()

# Round out our sums

top_20 = round(top_20)

# create a simple bar chart with labels

top_20.plot_bokeh(
    kind='bar',
    x='City',
    y=['Sum of Daily Mean AQIs'],
    xlabel='City',
    ylabel='Sum of Daily Mean AQIs',
    title='Sum of Total Mean AQIs Reported by City',
    figsize=(900,500),
    vertical_xlabel=True
)

Loading BokehJS ...

Figure(id='1038', ...)

In [19]:
# We can also make nice time series visualization

# First we group by day and sum up all values
aqi_by_day = aqi_mean_df.groupby(['Parsed Date'])['Mean AQI by Day'].sum().reset_index()

# Then we establish a datetime index
aqi_by_day.set_index('Parsed Date', inplace = True)

# Next we round our values
aqi_by_day = round(aqi_by_day)

# Finally, we plot w/ rangetool
aqi_by_day.plot_bokeh(rangetool=True)

Column(id='1229', ...)

In [ ]:
# We can also drill down to look at a specific city and see how much day over day change there is.

aqi_mean_df.set_index('Parsed Date', inplace = True)
aqi_mean_df['pct_change'] = aqi_mean_df.groupby(['City'])['Mean AQI by Day'].pct_change()

In [22]:
beijing_values = aqi_mean_df[aqi_mean_df['City'] == '北京市']

In [30]:
beijing_values = round(beijing_values, 2)

In [31]:
beijing_values.plot_bokeh(rangetool=True)

Column(id='2088', ...)